In [131]:
import pandas as pd # для обработки и анализа данных
import numpy as np # для математических вычислений
from tqdm.auto import tqdm # для отслеживания прогресса длительных операций
import matplotlib.pyplot as plt # для визуализации
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from string import punctuation
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.models import Word2Vec
import gensim.downloader as api
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
from sklearn.manifold import TSNE
import re
from typing import List, Set
import gensim
from tqdm import tqdm
from multiprocessing import Pool, cpu_count


In [4]:
re.sub(pattern=r'<.*>', repl='replace_this', string='<a> b <c>')
re.sub(pattern=r'<.*?>', repl='replace_this', string='<a> b <c>')

'replace_this b replace_this'

In [12]:
s = '111'
print(s)

111


In [ ]:
re.sub(pattern=r'\{\{.*?\}\}', repl='', string='{{111}}12345{{000}}trtrt')

'12345trtrt'

In [2]:
def remove_private_data(texts: List[str]) -> List[str]:
    return [re.sub(pattern=r'\{\{.*?\}\}', repl='', string=x) for x in texts]

In [5]:
texts : List[str] = ['{{111}}111{222}}aaa', '222', '333']
remove_private_data(texts)

['111{222}}aaa', '222', '333']

In [20]:
pattern_str = '[' + re.escape(punctuation) + '0123456789' + ']'
re.sub(pattern=pattern_str, repl='', string='aaa123.{]1bbb.87ccc')

'aaabbbccc'

In [21]:
def remove_nonalpha(texts: List[str]) -> List[str]:
    pattern_str = '[' + re.escape(punctuation) + '0123456789' + ']'
    return [re.sub(pattern=pattern_str, repl='', string=x) for x in texts]

In [24]:
texts : List[str] = ['{{111}}111{222}}aaa', '222gjhg897.,686', '333']
remove_nonalpha(texts)

['aaa', 'gjhg', '']

In [29]:
def lemmatize_corpus(texts: List[str]) -> List[List[str]]:
    lemmatizer = WordNetLemmatizer()
    return [[lemmatizer.lemmatize(y) for y in re.compile(r'\w+').findall(x.lower())] for x in texts]

In [35]:
texts : List[str] = ['dogs cat worked', '12345 678', '333']
lemmatize_corpus(texts)

[['dog', 'cat', 'worked'], ['12345', '678'], ['333']]

In [44]:
stopwords.words('english') + list(gensim.parsing.preprocessing.STOPWORDS)

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [64]:
def remove_stopwords(texts, stopwords):
    return [[y for y in re.compile(r'\w+').findall(x) if y not in stopwords] for x in texts]

In [55]:
def remove_stopwords(texts: List[str], stopwords: List[str]) -> List[List[str]]:
    return [[y for y in re.compile(r'\w+').findall(x) if y not in stopwords] for x in texts]

In [65]:
texts : List[str] = ['dogs my worked', 'down 678', '333']
stopwords_list = stopwords.words('english') + list(gensim.parsing.preprocessing.STOPWORDS)
remove_stopwords(texts, stopwords_list)

[['dogs', 'worked'], ['678'], ['333']]

In [136]:
from nltk.metrics.distance import edit_distance
import nltk
nltk.download('words')

all_words = nltk.corpus.words.words()

def preprocess(words):
    fixed_words = []
    for word in words:
        if word in all_words:
            fixed_words.append(word)
        else:
            result = [
                (edit_distance(word, w), w)
                for w in all_words
                if w[0] == word[0] and w[-1] == word[-1] and abs(len(w) - len(word)) < 3
            ]
            if len(result) == 0:
                continue
            distances, words = zip(*result)
            fixed_words.append(words[np.argmin(distances)])

    return fixed_words


def fix_typos(texts):
    # было [preprocess(text) for text in texts]
    return [preprocess(text) for text in tqdm(texts, desc="Обработка текстов", unit="текст")]

def fix_typos_parallel(texts, n_processes=None):    


    # Плоский список всех слов
    flat_words = [word for sublist in texts for word in sublist]


    # Количество процессов
    num_processes = n_processes or cpu_count()


    with Pool(num_processes) as pool:
        processed_flat = list(tqdm(pool.imap(preprocess, flat_words), total=len(flat_words), desc="Обработка слов"))

    # Восстановление структуры: список списков строк
    result = []
    idx = 0
    for sublist in texts:
        count = len(sublist)
        result.append(processed_flat[idx:idx + count])
        idx += count

    return result

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [135]:
texts =  [['doog', 'cat', 'workked'], ['12345', '678'], ['333']]
fix_typos(texts)
fix_typos_parallel(texts)

Обработка слов: 100%|██████████| 6/6 [00:00<00:00, 23.51it/s]


[[['d', 'o', 'o', 'g'], ['c', 'a', 't'], ['w', 'o', 'r', 'k', 'k', 'e', 'd']],
 [[], []],
 [[]]]

In [83]:
def write_nested_list_to_file(filename, nested_list):
    """
    Записывает список списков строк в файл, преобразуя в плоский список строк
    
    Параметры:
        filename (str): Имя файла для записи
        nested_list (list): Список списков строк (например, [["a","b"], ["c"]])
    
    Возвращает:
        list: Плоский список всех строк (например, ["a", "b", "c"])
    """
    # Преобразуем вложенный список в плоский
    flat_list = [item for sublist in nested_list for item in sublist]
    
    # Записываем в файл
    with open(filename, 'w', encoding='utf-8') as f:
        for line in flat_list:
            f.write(line + '\n')
    
    return flat_list

In [95]:
def split_file_to_words(filename):
    """
    Читает файл и возвращает список списков слов (каждая строка разбивается по пробелам)
    
    Args:
        filename (str): путь к входному файлу
        
    Returns:
        list: список списков слов, например:
             [['help', 'cageling', 'purchase'],
              ['help', 'cageling', 'purchase']]
    """
    result = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:  # если строка не пустая
                result.append(line.split())  # разбиваем по пробелам
    return result

In [102]:
texts = split_file_to_words('data/step5.txt')
texts[1]

['question', 'canalling', 'order']

In [103]:
texts_fix_typos=fix_typos_parallel(texts)

Обработка текстов: 100%|██████████| 26707/26707 [01:45<00:00, 252.68текст/s]


In [121]:
def count_unique_words(list_of_word_lists: List[List[str]]) -> int:
    return len({word for sublist in list_of_word_lists for word in sublist})

def set_unique_words(list_of_word_lists: List[List[str]]) -> Set[str]:
    return {word for sublist in list_of_word_lists for word in sublist}

In [119]:
texts[10000]

['need', 'assistance', 'open', 'account', 'dad']

In [122]:
set_unique_words(texts_fix_typos)

{'seed',
 'nay',
 'remember',
 'close',
 'theaterless',
 'teinland',
 'astarboard',
 'referribleness',
 'triform',
 'constat',
 'matross',
 'waar',
 'hello',
 'request',
 'older',
 'ope',
 'odorant',
 'ocellation',
 'hep',
 'calves',
 'obligement',
 'stolen',
 'ice',
 'shaping',
 'apeiron',
 'thonder',
 'shaw',
 'cade',
 'opinionist',
 'edit',
 'thanage',
 'may',
 'dot',
 'the',
 'husband',
 'penalty',
 'lit',
 'write',
 'arrie',
 'restitutionism',
 'chanting',
 'perhaps',
 'saj',
 'youve',
 'aln',
 'fro',
 'transcribe',
 'indrawal',
 'elbow',
 'war',
 'unhappy',
 'iceleaf',
 'acanthin',
 'troublous',
 'many',
 'ubi',
 'septemvir',
 'looking',
 'grotto',
 'semilichen',
 'apical',
 'sannup',
 'service',
 'unaccept',
 'customer',
 'declined',
 'howso',
 'calcaneal',
 'candle',
 'toi',
 'loan',
 'touse',
 'constancy',
 'cloying',
 'acquire',
 'mew',
 'wain',
 'aseismatic',
 'yallow',
 'opinion',
 'extricated',
 'damn',
 'sole',
 'inwith',
 'myth',
 'anadem',
 'yar',
 'ideate',
 'cadew',
 

In [127]:
from typing import List, Tuple, Dict, Set
from collections import defaultdict
import pprint

def compare_word_lists(
    list1: List[List[str]], 
    list2: List[List[str]]
) -> Dict[str, float]:
    """
    Сравнивает два списка списков слов и возвращает метрики сравнения.
    
    Args:
        list1: Первый список списков слов
        list2: Второй список списков слов
    
    Returns:
        Словарь с метриками сравнения:
        - common_lines: Количество одинаковых строк (списков слов)
        - common_words: Количество общих уникальных слов
        - word_overlap: Процент совпадения слов (0-1)
        - jaccard_similarity: Коэффициент Жаккара для множеств слов
        - line_position_diff: Разница позиций общих строк
    """
    # Преобразуем списки слов в кортежи для хеширования
    tuples1 = [tuple(sublist) for sublist in list1]
    tuples2 = [tuple(sublist) for sublist in list2]
    
    # Множества для сравнения
    set1 = set(tuples1)
    set2 = set(tuples2)
    
    # Уникальные слова
    words1 = {word for sublist in list1 for word in sublist}
    words2 = {word for sublist in list2 for word in sublist}
    
    # Позиции строк
    pos1 = {tpl: i for i, tpl in enumerate(tuples1)}
    pos2 = {tpl: i for i, tpl in enumerate(tuples2)}
    
    # Вычисление метрики
    common_lines = set1 & set2
    common_words = words1 & words2
    
    return {
        'common_lines_count': len(common_lines),
        'common_lines_examples': list(common_lines)[:3],  # Примеры
        'common_words_count': len(common_words),
        'common_words_examples': list(common_words)[:3],
        'word_overlap': len(common_words) / max(len(words1), len(words2), 1),
        'jaccard_similarity': len(common_words) / len(words1 | words2) if (words1 | words2) else 0,
        'line_position_diff': [abs(pos1[tpl]-pos2[tpl]) for tpl in common_lines],
        'unique_in_list1': len(set1 - set2),
        'unique_in_list2': len(set2 - set1)
    }

In [129]:
def compare_word_lists(
    list1: List[List[str]], 
    list2: List[List[str]]
) -> Tuple[int, int, int]:
    # Получаем множества уникальных слов для каждого списка
    words1: Set[str] = {word for sublist in list1 for word in sublist}
    words2: Set[str] = {word for sublist in list2 for word in sublist}
    
    # Вычисляем пересечение и уникальные слова
    common_words = words1 & words2
    unique1 = words1 - words2
    unique2 = words2 - words1
    
    return len(common_words), len(unique1), len(unique2)

In [124]:
text_metric_dict = compare_word_lists(texts, texts_fix_typos)

In [128]:
pprint.pp(text_metric_dict)

{'common_lines_count': 16238,
 'common_lines_examples': [('hour', 'can', 'contact', 'customer', 'support'),
                           ('receive', 'newsletter', 'help'),
                           ('assistance',
                            'sending',
                            'feedback',
                            'ur',
                            'podices')],
 'common_words_count': 1550,
 'common_words_examples': ['seed', 'nay', 'remember'],
 'word_overlap': 1.0,
 'jaccard_similarity': 1.0,
 'line_position_diff': [0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
              

In [130]:
compare_word_lists(texts, texts_fix_typos)

(1550, 0, 0)

In [137]:
def remove_rare_words(texts):
    vocab = set()
    for words in texts:
        vocab |= set(words)

    rare_words = []
    for word in vocab:
        count = sum([(word in text) for text in texts])
        if count < 4:
            rare_words.append(word)

    new_texts = []
    for text in texts:
        cleaned = [word for word in text if word not in rare_words]
        new_texts.append(cleaned)
    return new_texts

In [138]:
def train_w2v(texts):
    model = Word2Vec(
        texts,
        vector_size=100,
        min_count=0,
        window=3
    ).wv
    return model

In [139]:
def get_text_embedding(words, w2v_model):
    embeddings = np.array([w2v_model.get_vector(word) for word in words])
    embedding = np.mean(embeddings, axis=0)
    return embedding

In [ ]:
def find_best_k(text_embeddings):
    scores = []
    ks = np.arange(3, 30, 2)
    for k in ks:
        kmeans = KMeans(n_clusters=k)
        clusters = kmeans.fit_predict(text_embeddings)
        score = silhouette_score(text_embeddings, clusters)
        scores.append(score)

    return ks[np.argmax(scores)]